# Manifest Validation


In [34]:
import random
import string
import pandas as pd
from pathlib import Path
from glob import glob
import os

# Validation of manifests


In [2]:
# number of rows to fill in
nrows = 10
random.seed = 27

# last minute chaos
chaos = False

Functions


In [3]:
def valid_values_to_list(df, attribute):
    valid_values = (
        df.query(f'Attribute == "{attribute}"')[
            "Valid Values"].str.split(",").values[0]
    )

    return valid_values

In [4]:
def get_random_value(list_of_vv):
    return random.choice(list_of_vv)

In [5]:
def get_rand_integer(min=0, max=100):
    return random.randint(min, max)

In [6]:
def get_rand_float(min=0, max=100):
    return round(random.uniform(0.0, 100.0), 2)

In [7]:
def get_random_string():
    t = lorem.sentence().split(" ")[0]
    return t

In [8]:
def introduce_random_NAs(df, N=5):
    """Another test to see if columns can handle empty values or if they will flag the empty value"""

    rows, cols = df.shape

    row_index = [get_rand_integer(max=rows - 1) for _ in range(N)]

    col_index = [get_rand_integer(max=cols - 1) for _ in range(N)]

    indexes = list(zip(row_index, col_index))

    # Print indexes to check where values got replaced
    print(indexes)
    # for i in indexes
    df.iloc[row_index, col_index] = np.NaN

    return df

In [9]:
# find attribute column, fill in with value
def fill_in_attribute(df, index, attribute, value):
    df.loc[index, attribute] = value
    return df

In [10]:
def gen_mixed_string_with_length(N=12):
    # initializing size of string

    # using random.choices()
    # generating random strings
    res = "".join(random.choices(string.ascii_uppercase + string.digits, k=N))

    # print result
    return res

In [11]:
def random_change():
    # list of functions to choose from
    choices = [
        introduce_random_NAs,
        gen_mixed_string_with_length,
        get_rand_integer,
        get_random_string,
        get_rand_float,
    ]

    choice = random.choice(choices)

    print(choice.__name__)

    return choice()

In [12]:
def partition(list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

Create individual and biospecimen ids from random text


In [13]:
# for individaul IDs and specimenIDs
individaulIds = [gen_mixed_string_with_length(N=10) for _ in range(1, 100)]
specimenIds = [gen_mixed_string_with_length(N=5) for _ in range(1, 1000)]

In [14]:
specimenIds_partitioned = partition(specimenIds, 100)

In [15]:
ind_bio_map = []

for i, v in enumerate(individaulIds):
    for s in specimenIds_partitioned[i]:
        ind_bio_map.append({"individualID": v, "specimenID": s})

ind_bio_map = pd.DataFrame(ind_bio_map)

In [16]:
ind_bio_map.head(20)

,individualID,specimenID
0,JPXMA6RZ99,I4B0N
1,JPXMA6RZ99,WZFHW
2,JPXMA6RZ99,NC5TS
3,JPXMA6RZ99,YMGPO
4,JPXMA6RZ99,09G79
5,JPXMA6RZ99,N01CM
6,JPXMA6RZ99,1N4GW
7,JPXMA6RZ99,XN5X5
8,JPXMA6RZ99,I55AK
9,JPXMA6RZ99,K3VVB


In [20]:
# Manifests
# Get all the RFC file paths
manifest_paths = glob(
    "../manifests/*.csv"
)
manifest_paths

[]

Tested:

- scRNAseq : Passed (Need to add validation for biospecimenId)


In [21]:
# load data model
data_model_path = ("../EL.data.model.csv"
)

dm = pd.read_csv(data_model_path)

dm.head()

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Type,Ontology,UsedIn
0,Affymetrix Genome-Wide Human SNP 6.0 Array,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Assay,NaN,NaN,NaN
1,Affymetrix Human Gene 1.0 ST Array,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Assay,NaN,NaN,NaN
2,Affymetrix Human Genome U133 Plus 2.0 Array,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Assay,NaN,NaN,NaN
3,Affymetrix U133AB,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Assay,NaN,NaN,NaN
4,DNA,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Assay,NaN,NaN,NaN


In [22]:
# load manifest
manifest_path = "../example-scRNAseq.csv"

df = pd.read_csv(manifest_path)

df = df.dropna(how="all", axis=0)

df = df.fillna('')

df.head()

,Filename,specimenID,sampleType,specifySampleType,specifyMeasurementTechnique,technologyPlatformVersion,specifyPlatformVersion,platformLocation,specifyPlatformLocation,referenceTranscriptID,...,libraryVersion,specifyLibraryVersion,isStranded,readStrandOrigin,readLength,readLengthUnits,specifyReadLengthUnits,runType,totalReads,Component
0,,,,,,,,,,,...,,,,,,,,,,ScRNAseq


In [23]:
manifest_name = Path(manifest_path).stem

parent_name = df['Component'].unique()[0]

df["Component"] = manifest_name

In [24]:
# Attributes from data model
attrs_to_fill = df.columns.tolist()

attrs_to_fill

['Filename',
 'specimenID',
 'sampleType',
 'specifySampleType',
 'specifyMeasurementTechnique',
 'technologyPlatformVersion',
 'specifyPlatformVersion',
 'platformLocation',
 'specifyPlatformLocation',
 'referenceTranscriptID',
 'repositoryName',
 'specifyRepositoryName',
 'transcriptType',
 'specifyTranscriptType',
 'resultUnit',
 'valueReported',
 'rnaBatchID',
 'rnaBatchSize',
 'rnaBatchSizeUnit',
 'specifyRNABatchSizeUnit',
 'libraryBatchID',
 'sequencingBatchID',
 'sequencingBatchSize',
 'sequencingBatchSizeUnit',
 'specifySequencingBatchSizeUnit',
 'libraryPrep',
 'specifyLibraryPrep',
 'libraryPreparationMethod',
 'specifyLibraryPreparationMethod',
 'libraryVersion',
 'specifyLibraryVersion',
 'isStranded',
 'readStrandOrigin',
 'readLength',
 'readLengthUnits',
 'specifyReadLengthUnits',
 'runType',
 'totalReads',
 'Component']

In [25]:
dm_attrs = (
    dm[(dm["Attribute"].isin(attrs_to_fill))]
    .drop_duplicates(subset=["Attribute"])
    .copy()
)

dm_attrs

,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Type,Ontology,UsedIn
16,isStranded,Whether or not the library is stranded.,"TRUE,FALSE",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,str,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
17,libraryBatchID,"Library batch identifier, provided by the data...",NaN,NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,Whole Genome Sequencing,bsSeq,scRNAseq"
18,libraryPrep,The general strategy by which the library was ...,"amplicon,cellHashing,Chromium Single Cell 3',D...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,Whole Genome Sequencing,bsSeq,scRNAseq"
19,libraryPreparationMethod,Method by which library was prepared,"10x,Accel-NGS 2S Plus,Accel-NGS Methyl-Seq,CEL...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
20,libraryVersion,"Library Version: for example, rnaSeq 10x libra...","OtherLibraryVersion,Unknown,Not collected,Not ...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
21,platformLocation,"The name of the laboratory, facility, vendor, ...","OtherPlatformLocation,Unknown,Not collected,No...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Metabolomics Human,Microbiome,RNAseq,Whole Gen..."
25,readLength,The length of the read,NaN,NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,regex search ([0-9]+\.[0-9]*.)|([0-9]+),Assay,NaN,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
26,readLengthUnits,The units of measurement.,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
27,readStrandOrigin,The strand from which the read originates in a...,"forward,reverse",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"Microbiome,RNAseq,Whole Genome Sequencing,bsSe..."
35,referenceTranscriptID,The ID for the transcript/gene. Either the NCB...,"Unknown,Not collected,Not applicable,Not Speci...",NaN,NaN,True,DataProperty,NaN,Sage Bionetworks,NaN,Assay,NaN,NaN,"RNAseq,Whole Genome Sequencing,bsSeq,scRNAseq"


In [26]:
# find values in dm_attrrs with valid values filled in
vv_attrs = dm_attrs[~dm_attrs["Valid Values"].isna()]["Attribute"].tolist()

In [27]:
vv_attrs

['isStranded',
 'libraryPrep',
 'libraryPreparationMethod',
 'libraryVersion',
 'platformLocation',
 'readLengthUnits',
 'readStrandOrigin',
 'referenceTranscriptID',
 'rnaBatchID',
 'rnaBatchSizeUnit',
 'runType',
 'sequencingBatchID',
 'sequencingBatchSizeUnit',
 'technologyPlatformVersion',
 'repositoryName',
 'resultUnit',
 'sampleType',
 'transcriptType']

In [28]:
# fill in values for the mainfest with the data model
for attribute in vv_attrs:
    for i in range(nrows):
        index = i
        temp_vv = valid_values_to_list(dm_attrs, attribute)
        new_val = get_random_value(temp_vv)
        df = fill_in_attribute(df, index, attribute, new_val)

In [29]:
df

,Filename,specimenID,sampleType,specifySampleType,specifyMeasurementTechnique,technologyPlatformVersion,specifyPlatformVersion,platformLocation,specifyPlatformLocation,referenceTranscriptID,...,libraryVersion,specifyLibraryVersion,isStranded,readStrandOrigin,readLength,readLengthUnits,specifyReadLengthUnits,runType,totalReads,Component
0,,,Lung lymph node,,,Vectra H1 3D Imaging System,,Not applicable,,Not applicable,...,Not collected,,TRUE,reverse,,AU/ml,,singleEnd,,example-scRNAseq
1,NaN,NaN,Bone,NaN,NaN,Not applicable,NaN,OtherPlatformLocation,NaN,Unknown,...,Not collected,NaN,FALSE,forward,NaN,pg/ml,NaN,pairedEnd,NaN,NaN
2,NaN,NaN,Urinary bladder,NaN,NaN,Unknown,NaN,Not Specified,NaN,Not collected,...,OtherLibraryVersion,NaN,TRUE,forward,NaN,Pound,NaN,pairedEnd,NaN,NaN
3,NaN,NaN,Dendritic cell,NaN,NaN,Illumina NextSeq 1000,NaN,OtherPlatformLocation,NaN,Not Specified,...,Not collected,NaN,TRUE,reverse,NaN,Not Specified,NaN,pairedEnd,NaN,NaN
4,NaN,NaN,OtherSampleType,NaN,NaN,Nanostring GeoMx,NaN,OtherPlatformLocation,NaN,Not applicable,...,Not collected,NaN,TRUE,forward,NaN,HAU,NaN,pairedEnd,NaN,NaN
5,NaN,NaN,brain,NaN,NaN,Illumina HumanOmniExpress-24 v1.0 BeadChip,NaN,Not applicable,NaN,Not Specified,...,OtherLibraryVersion,NaN,FALSE,reverse,NaN,ug/dl,NaN,singleEnd,NaN,NaN
6,NaN,NaN,Mouse Ear,NaN,NaN,Vevo 3100 Imaging System,NaN,Not applicable,NaN,Unknown,...,Not Specified,NaN,FALSE,forward,NaN,optical density,NaN,singleEnd,NaN,NaN
7,NaN,NaN,Basophils,NaN,NaN,Illumina HumanHap300,NaN,OtherPlatformLocation,NaN,Not applicable,...,Unknown,NaN,FALSE,forward,NaN,iu/l,NaN,singleEnd,NaN,NaN
8,NaN,NaN,Inguinal lymph node,NaN,NaN,scale,NaN,Not collected,NaN,Not applicable,...,OtherLibraryVersion,NaN,TRUE,forward,NaN,pM,NaN,pairedEnd,NaN,NaN
9,NaN,NaN,PBL,NaN,NaN,Illumina NovaSeq 6000,NaN,Not Specified,NaN,Not collected,...,Not collected,NaN,TRUE,forward,NaN,ml,NaN,singleEnd,NaN,NaN


In [30]:
# update df with sample of individual and biospecimen sample ids
ind_sample = ind_bio_map.sample(nrows, replace=True)[
    ["individualID", "specimenID"]
].reset_index(drop=True)

df.update(ind_sample)

In [32]:
df

,Filename,specimenID,sampleType,specifySampleType,specifyMeasurementTechnique,technologyPlatformVersion,specifyPlatformVersion,platformLocation,specifyPlatformLocation,referenceTranscriptID,...,libraryVersion,specifyLibraryVersion,isStranded,readStrandOrigin,readLength,readLengthUnits,specifyReadLengthUnits,runType,totalReads,Component
0,,6VTXG,Lung lymph node,,,Vectra H1 3D Imaging System,,Not applicable,,Not applicable,...,Not collected,,TRUE,reverse,,AU/ml,,singleEnd,,example-scRNAseq
1,NaN,4Z2UM,Bone,NaN,NaN,Not applicable,NaN,OtherPlatformLocation,NaN,Unknown,...,Not collected,NaN,FALSE,forward,NaN,pg/ml,NaN,pairedEnd,NaN,NaN
2,NaN,2ID8X,Urinary bladder,NaN,NaN,Unknown,NaN,Not Specified,NaN,Not collected,...,OtherLibraryVersion,NaN,TRUE,forward,NaN,Pound,NaN,pairedEnd,NaN,NaN
3,NaN,NESZ5,Dendritic cell,NaN,NaN,Illumina NextSeq 1000,NaN,OtherPlatformLocation,NaN,Not Specified,...,Not collected,NaN,TRUE,reverse,NaN,Not Specified,NaN,pairedEnd,NaN,NaN
4,NaN,ZBJSB,OtherSampleType,NaN,NaN,Nanostring GeoMx,NaN,OtherPlatformLocation,NaN,Not applicable,...,Not collected,NaN,TRUE,forward,NaN,HAU,NaN,pairedEnd,NaN,NaN
5,NaN,UTZ9D,brain,NaN,NaN,Illumina HumanOmniExpress-24 v1.0 BeadChip,NaN,Not applicable,NaN,Not Specified,...,OtherLibraryVersion,NaN,FALSE,reverse,NaN,ug/dl,NaN,singleEnd,NaN,NaN
6,NaN,27S1P,Mouse Ear,NaN,NaN,Vevo 3100 Imaging System,NaN,Not applicable,NaN,Unknown,...,Not Specified,NaN,FALSE,forward,NaN,optical density,NaN,singleEnd,NaN,NaN
7,NaN,5P83Y,Basophils,NaN,NaN,Illumina HumanHap300,NaN,OtherPlatformLocation,NaN,Not applicable,...,Unknown,NaN,FALSE,forward,NaN,iu/l,NaN,singleEnd,NaN,NaN
8,NaN,LF97B,Inguinal lymph node,NaN,NaN,scale,NaN,Not collected,NaN,Not applicable,...,OtherLibraryVersion,NaN,TRUE,forward,NaN,pM,NaN,pairedEnd,NaN,NaN
9,NaN,LC9BR,PBL,NaN,NaN,Illumina NovaSeq 6000,NaN,Not Specified,NaN,Not collected,...,Not collected,NaN,TRUE,forward,NaN,ml,NaN,singleEnd,NaN,NaN


In [31]:
swap_validation_coder = {v: k for k, v in validation_coder.items()}

# column coding for values
dm_attrs["swapper"] = (
    dm_attrs["Validation Rules"].fillna(
        "number").replace(swap_validation_coder)
)

dm_attrs["swapper"] = dm_attrs["swapper"].apply(
    lambda x: re.sub("^\d*?\.?\d$", "number", x)
)

# need a better regex later
dm_attrs["swapper"] = dm_attrs["swapper"].apply(
    lambda x: re.sub(
        "number numbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumbernumber",
        "number",
        x,
    )
)

print(dm_attrs["swapper"].unique())

dm_attrs.head()

NameError: name 'validation_coder' is not defined

In [ ]:
free_form_attrs = dm_attrs[
    (dm_attrs["Valid Values"] == "")
    & (~dm_attrs["Attribute"].isin(["individualID", "specimenID"]))
]

free_form_attrs

In [ ]:
def generate_values(value):
    if value == 'string':
        # generate string
        return get_random_string()
    elif value == 'number':
        # generate random number
        return get_rand_integer()
    elif value == 'mixed'
        # split string into regex expressions i.e. numbers and valid values
        choices = ['string', 'number']
        # chose random to fill in cell

In [ ]:
test = "([0-9]+\\.[0-9]*.)|([0-9]+)"
test2 = "^\\d*?\\.?\\d$|Other|Unknown|Not collected|Not applicable"

# def match_pattern(rule):

pattern_checker = {
    "strip_patterns": "regex|search|match",
    "number": "(?![a-zA-Z]+)(\[0-9\])",
    "string": "([a-zA-Z]+)(?!\[0-9\])",
    "mixed": "(?:[a-zA-Z]+\d+)",
}

result = False

while result == False:
    for k, v in pattern_checker.items():
        result = bool(re.search(pattern=v, string=test))
        print(pattern)
        print(result)


# pattern = '(?![a-zA-Z]+)(\[0-9\])'
# results = bool(re.search(pattern = pattern, string = test))
# results

In [ ]:
for attribute in free_form_attrs["Attribute"].tolist():
    for i in range(nrows):
        index = i
        test = free_form_attrs.query(f'Attribute == "{attribute}"')
        new_val = test["swapper"].apply(generate_values).values[0]
        df = fill_in_attribute(df, index, attribute, new_val)

In [ ]:
# write out test manifest to file for testing in DCA
df["Component"] = manifest_name

if chaos:
    df = introduce_random_NAs(df)

In [36]:
# Write out manifest
csv_path = os.path.join(
    "../testing/test_manifests",
    manifest_name + "-test.csv",
)

df.to_csv(csv_path)

# Run Validation Test


In [ ]:
# manifest_path = "C:/Users/nlee/Documents/Projects/schematic/schematic/tests/data/mock_manifests/example_biospecimen_test.csv"
print(
    f"schematic model --config {schematic_config} validate --manifest_path {csv_path} --data_type {manifest_name}")
!schematic model --config {schematic_config} validate --manifest_path {csv_path} --data_type {manifest_name}

# Submit Manifest


In [ ]:
print(csv_path)
print(schematic_config)
print(manifest_name)
dataset_id = "syn51753850"

- d option is the project


In [ ]:
dataset_id = 'syn51753844'

!schematic model --config C:/Users/nlee/Documents/Projects/schematic/schematic/config.yml submit -mp C:/Users/nlee/Documents/Projects/ELITE-DCC/ELITE-data-models/test_manifests/WholeGenomeSequencing_test.csv -d syn51753850 -vc WholeGenomeSequencing -mrt both

In [ ]:
search_df(dm2, "valueReported")

# Store Manifest In Dataset For Testing


In [ ]:
import synapseclient
from synapseclient import File

manifest_folder_id = 'syn51728840'

%run C:\Users\nlee\Documents\Projects\utils\syanpse_login.py


In [ ]:
csv_entity = File(
    csv_path,
    description=f"Test manifest for {manifest_name}",
    parent=data_folder,
    annotations={"resourceType": "manifest", "manifestType": {manifest_name}},
)

csv_entity = syn.store(csv_entity)

In [ ]:
s = syn.getColumns("syn51748558")

In [ ]:
for i in s:
    print(i)